## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2024-11-28 08:42:59--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/pkkpoxlm7beasryexpdf8/data_000637.txt?rlkey=rkm2em1v57hewglzelmin21c9 [following]
--2024-11-28 08:42:59--  https://www.dropbox.com/scl/fi/pkkpoxlm7beasryexpdf8/data_000637.txt?rlkey=rkm2em1v57hewglzelmin21c9
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4c652649efef8e0ba69bb0c141.dl.dropboxusercontent.com/cd/0/inline/CfNCzIZD8PhacJ8tziGNIItCnKHnqKNeLkPWQWx63opNAYqWDnp54Rf4xQ2c3gqnvWmQklziIGMOBFxaPvL3RQ9ia8Pf7QvEdBq8mzTXAgcLuHucrHZnpl1IaxOtyy29JSk/file# [following]
--2024-11-28 08:43:00--  https://uc4c652649efef8e0ba69bb0c141.dl.dropboxusercontent.com/cd/0/inline/CfNCzIZD8P

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [27]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/ungureav/data/data_000637.txt.1', nrows = 10001)
display(df[:100])

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
5,1,0,63,3869200167,2762,4
6,1,0,61,3869200167,2772,14
7,1,0,139,3869200167,2776,0
8,1,0,62,3869200167,2774,21
9,1,0,60,3869200167,2788,7


In [10]:
filtered_data = df.loc[df['ORBIT_CNT'] == 3869200167, 'BX_COUNTER']
#command for newer versions:filtered_data = filtered_data.to_numpy()
#command for older versions:
filtered_data = filtered_data.values
sub = filtered_data[-1] - filtered_data[0]
print(sub)

813


In [14]:
orbit_sub = (df['ORBIT_CNT'].values[-1] - df['ORBIT_CNT'].values[0])* 25 * 813
bx_sub = (df['BX_COUNTER'].values[-1] - df['BX_COUNTER'].values[0]) * 25
tdc_sub = (df['BX_COUNTER'].values[-1] - df['BX_COUNTER'].values[0]) * 25/30
print(orbit_sub + bx_sub + tdc_sub , 'ns')

1991218.3333333333 ns


In [19]:
TIME = []
for i in range(len(df['ORBIT_CNT'].values)):
    time = df['ORBIT_CNT'].values[i] * 25 *813 + df['BX_COUNTER'].values[i] * 25 + df['TDC_MEAS'].values[i] * 25/30
    TIME.append(time)
df['ABS_TIME'] = TIME
df.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
0,1,0,123,3869200167,2374,26,7.864149e+13
1,1,0,124,3869200167,2374,27,7.864149e+13
2,1,0,63,3869200167,2553,28,7.864149e+13
3,1,0,64,3869200167,2558,19,7.864149e+13
4,1,0,64,3869200167,2760,25,7.864149e+13


In [22]:
df['HEAD'] = np.random.choice([0,1], size = df.shape[0])
df.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
0,1,0,123,3869200167,2374,26,7.864149e+13
1,0,0,124,3869200167,2374,27,7.864149e+13
2,1,0,63,3869200167,2553,28,7.864149e+13
3,0,0,64,3869200167,2558,19,7.864149e+13
4,1,0,64,3869200167,2760,25,7.864149e+13


In [24]:
new_data = df.loc[df['HEAD'] == 1]
new_data.head()

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
0,1,0,123,3869200167,2374,26,7.864149e+13
2,1,0,63,3869200167,2553,28,7.864149e+13
4,1,0,64,3869200167,2760,25,7.864149e+13
5,1,0,63,3869200167,2762,4,7.864149e+13
7,1,0,139,3869200167,2776,0,7.864149e+13


In [26]:
new_data = new_data.groupby('TDC_CHANNEL')
new_data.head()
#for i in range(len(df['HEAD'].values)):
    
    

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method